# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, warnings, random, string
from pickle import dump, load
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from keras.applications.inception_v3 import InceptionV3 , preprocess_input
from keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

In [ ]:
from nltk import FreqDist
from nltk.translate.bleu_score import sentence_bleu

# Loading Dataset and Performing EDA

In [ ]:
input_dir = '../input/flickr8k/'


In [ ]:
def load_captions_dictionary(path):
    file = open(path, 'r')
    captions = file.read().split('\n')
    descriptions = {}
    for text in captions[1:]:
        values = text.split(',')
        img, caption = values[0].split('.')[0], "".join(values[1:])
        if img not in descriptions:
            descriptions[img] = [caption]
        else:
            descriptions[img].append(caption)
    file.close()
    return descriptions

descriptions = load_captions_dictionary(input_dir + 'captions.txt')

In [ ]:
npic = 5
img_size = 299
target_size = (img_size, img_size)
path = input_dir + "Images/"
fig = plt.figure(figsize=(10,20))

count = 1
for img in os.listdir(path)[:npic]:
    
    filename = path + img
    captions = list(descriptions[img.split(".")[0]])
    image_load = load_img(filename, target_size=target_size)
    
    ax = fig.add_subplot(npic, 2, count, xticks=[], yticks=[])
    ax.imshow(image_load)
    count += 1
    
    ax = fig.add_subplot(npic, 2, count)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, len(captions))
    for i, caption in enumerate(captions):
        ax.text(0, i, caption, fontsize=20)
    count += 1
plt.show()

# Text Cleaning and Processing

In [ ]:
def text_cleaning(descriptions):
    table = str.maketrans('', '', string.punctuation)
    for img, caption in descriptions.items():
        for i, img_text in enumerate(caption):
            img_text.replace("-", " ")
            text = [word.lower() for word in img_text.split()]
            text = [word.translate(table) for word in text]
            text = [word for word in text if(len(word) > 1)]
            text = [word for word in text if(word.isalpha())]
            img_text = " ".join(text)
            descriptions[img][i] = img_text
    return descriptions

descriptions = text_cleaning(descriptions)

In [ ]:
def corpus_and_vocab(descriptions):
    corpus = ""
    for img_text in descriptions.values():
        for text in img_text:
            corpus += " "+text
    vocab = set(corpus.split())
    return corpus, vocab

corpus, vocab = corpus_and_vocab(descriptions)
print("Number of unique words = {}".format(len(vocab)))
# Number of unique words being displayed

In [ ]:
freq_dist = FreqDist(corpus.split())
dfsub = pd.DataFrame(columns = ["word", "count"])
most_common = freq_dist.most_common()
words, counts = [], []
for i in range(len(freq_dist)):
    words.append(most_common[i][0])
    counts.append(most_common[i][1])
dfsub["word"], dfsub["count"] = words, counts

In [ ]:
def plthist(dfsub, title):
    plt.figure(figsize=(20,3))
    plt.bar(dfsub.index,dfsub["count"])
    plt.yticks(fontsize=20)
    plt.xticks(dfsub.index,dfsub["word"],rotation=90,fontsize=20)
    plt.title(title,fontsize=20)
    plt.show()
    
plthist(dfsub.iloc[:50], "The top 50 most frequently appearing words")
plthist(dfsub.iloc[-50:], "The least 50 most frequently appearing words")

In [ ]:
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for i, desc in enumerate(desc_list):
            descriptions[key][i] = desc = "<start> " + desc + " <end>"
            lines.append(key + '\t' + desc)
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()
    
save_descriptions(descriptions, "Image_Descriptions_List.txt")
# Putting start and end tags at the beginning and ending of each caption

# Train_Test_Val Split
#### First 10 images and captions are for the Test set
#### The next 10 are for the Validation set
#### All of the remaining images and captions are for the Training set

In [ ]:
df_img_caption = pd.DataFrame()
df_img_caption['Image_Name'] = list(descriptions.keys())[:-1]
temps = list(descriptions.values())[:-1]
df_img_caption['Caption'] = [temps[i][random.randint(0,4)] for i in range(len(temps))]
df_img_caption.head()

In [ ]:
test_images = np.asarray(df_img_caption['Image_Name'][:10], dtype = np.dtype(object))
test_captions = np.asarray(df_img_caption['Caption'][:10], dtype = np.dtype(object))
val_images = np.asarray(df_img_caption['Image_Name'][10:20], dtype = np.dtype(object))
val_captions = np.asarray(df_img_caption['Caption'][10:20], dtype = np.dtype(object))
train_images = np.asarray(df_img_caption['Image_Name'][20:], dtype = np.dtype(object))
train_captions = np.asarray(df_img_caption['Caption'][20:], dtype = np.dtype(object))

# Feature Extraction
#### Using the InceptionV3 model

In [ ]:
cnn_model = InceptionV3(weights = 'imagenet')
for layer in cnn_model.layers:
    layer.trainable = False
cnn_model = Model(inputs = cnn_model.input, outputs = cnn_model.get_layer('avg_pool').output)
cnn_model.summary()

In [ ]:
# Define Function to extract features
def extract_features(model, images, img_size):
    features = {}
    for img in tqdm(images):
        picture = load_img(input_dir + "Images/" + img + ".jpg", target_size = (img_size, img_size))
        picture = img_to_array(picture)
        picture = np.expand_dims(picture, axis = 0)
        picture = preprocess_input(picture)
        features[img] = model.predict(picture).reshape(2048,)
    
    return features

In [ ]:
# Training set features
Xtrain_features = extract_features(cnn_model, train_images, 299)

In [ ]:
# Validation set features
Xval_features = extract_features(cnn_model, val_images, 299)

In [ ]:
# Save features as an array
Xtrain_features = np.asarray(list(Xtrain_features.values()))
Xval_features = np.asarray(list(Xval_features.values()))

# Data Preparation

In [ ]:
#Split up captions into smaller parts
tokenizer = Tokenizer(num_words = len(vocab))
tokenizer.fit_on_texts(df_img_caption['Caption'])
word_to_index = tokenizer.word_index
index_to_word = dict([index, word] for word, index in word_to_index.items())
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_captions)
val_sequences = tokenizer.texts_to_sequences(val_captions)

def maxLength(sequences):
    return np.max([len(sequence) for sequence in sequences])

max_len = max(maxLength(train_sequences), maxLength(val_sequences))

In [ ]:
#Define Function for data to be generated so as to be fed into the rnn model
def data_generator(features, sequences):
    X_features, X_train, y_train = [], [], []
    for sequence, feature in zip(sequences, features):
        for i in range(1, len(sequence)):
            in_text, out_text = sequence[:i], sequence[i:]
            in_text = pad_sequences([in_text], maxlen = max_len)[0]
            out_text = to_categorical(out_text, num_classes = vocab_size)[0]
            X_features.append(feature)
            X_train.append(in_text)
            y_train.append(out_text)
    return (np.array(X_features), np.array(X_train), np.array(y_train))

Xt_features, Xt_text, yt_text = data_generator(Xtrain_features, train_sequences)
Xv_features, Xv_text, yv_text = data_generator(Xval_features, val_sequences)

In [ ]:
print(Xt_features.shape, Xt_text.shape, yt_text.shape)
print(Xv_features.shape, Xv_text.shape, yv_text.shape)

# RNN Model

In [ ]:
def define_model(vocab_size, max_len):

    inputs1 = Input(shape = (2048,))
    x1 = Dropout(0.3)(inputs1)
    x2 = Dense(256, activation = 'relu')(x1)

    inputs2 = Input(shape = (max_len,))
    se1 = Embedding(vocab_size, 256, mask_zero = True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    decoder1 = add([x2, se3])
    decoder2 = Dense(256, activation = 'relu')(decoder1)
    outputs = Dense(vocab_size, activation = 'softmax')(decoder2)

    rnn_model = Model(inputs = [inputs1, inputs2], outputs = outputs)
    rnn_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

    print(rnn_model.summary())
    plot_model(rnn_model, to_file = 'rnn_model.png', show_shapes = True)

    return rnn_model

rnn_model = define_model(vocab_size, max_len)

In [ ]:
model_training = rnn_model.fit([Xt_features, Xt_text], yt_text, 
                               epochs = 5, verbose = 2, batch_size = 64, 
                               validation_data = ([Xv_features, Xv_text], yv_text))

In [ ]:
for label in ["loss", "val_loss"]:
    plt.plot(model_training.history[label], label = label)
plt.legend()
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
# saving models
rnn_model.save("RNN_Model.h5")
cnn_model.save("CNN_Model.h5")
dump(tokenizer, open('Flickr8K_Tokenizer.p', 'wb'))

# Performance Analysis

In [ ]:
def generate_caption(filename):
    img = load_img(filename, target_size = (299, 299))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img)
    features = cnn_model.predict(img)
    in_text = 'start'
    for i in range(max_len):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_len)
        pred = rnn_model.predict([features,sequence], verbose=0)
        pred = np.argmax(pred)
        word = index_to_word[pred]
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [ ]:
def generate_caption_beam_search(filename, max_length, beam_index):
    img = load_img(filename, target_size = (299, 299))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = preprocess_input(img)
    features = cnn_model.predict(img)
    in_text = [[tokenizer.texts_to_sequences(['start'])[0], 0.0]]
    while len(in_text[0][0]) < max_length:
        tempList = []
        for seq in in_text:
            padded_seq = pad_sequences([seq[0]], maxlen=max_length)
            preds = rnn_model.predict([features,padded_seq], verbose=0)
            top_preds = np.argsort(preds[0])[-beam_index:]
            for word in top_preds:
                next_seq, prob = seq[0][:], seq[1]
                next_seq.append(word)
                prob += preds[0][word]
                tempList.append([next_seq, prob])
        in_text = tempList
        in_text = sorted(in_text, reverse=False, key=lambda l: l[1])
        in_text = in_text[-beam_index:]
    in_text = in_text[-1][0]
    final_caption_raw = [index_to_word[i] for i in in_text]
    final_caption = []
    for word in final_caption_raw:
        if word == 'end':
            break
        else:
            final_caption.append(word)
    final_caption.append('end')
    return ' '.join(final_caption)

In [ ]:
scores, beam2_scores, beam3_scores = [], [], []
for img, caption in zip(test_images, test_captions):
    hypothesis = generate_caption(input_dir + "Images/" + img + ".jpg")
    scores.append(sentence_bleu([caption.split()], hypothesis.split()))
    hypothesis = generate_caption_beam_search(input_dir + "Images/" + img + ".jpg", max_len, 2)
    beam2_scores.append(sentence_bleu([caption.split()], hypothesis.split()))
    hypothesis = generate_caption_beam_search(input_dir + "Images/" + img + ".jpg", max_len, 3)
    beam3_scores.append(sentence_bleu([caption.split()], hypothesis.split()))
    
for i, score in enumerate([scores, beam2_scores, beam3_scores]):
    plt.plot(score, label = 'Beam Length = ' + str(i+1))
plt.legend(loc = 'lower right')
plt.xlabel('Images')
plt.ylabel('Bleu Score')
plt.show()

In [ ]:
npic = 10
npix = 299
target_size = (npix,npix,3)

count = 1
fig = plt.figure(figsize=(20,20))
for img, true_caption in zip(test_images, test_captions):
     
    filename = input_dir + 'Images/' + img + ".jpg"
    image_load = load_img(filename, target_size = target_size)
    ax = fig.add_subplot(npic, 2, count, xticks=[], yticks=[])
    true_caption = ' '.join(true_caption.split()[1: -1])
    ax.imshow(image_load)
    count += 1

    caption = generate_caption(filename)
    caption = ' '.join(caption.split()[1: -1])
    ax = fig.add_subplot(npic, 2, count)
    plt.axis('off')
    ax.plot()
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.text(0, 0.7, true_caption, fontsize = 20)
    ax.text(0, 0.4, caption, fontsize = 20)
    ax.text(0, 0.1, 'Bleu Score = {}'.format(sentence_bleu([true_caption.split()], caption.split())), fontsize = 20)
    count += 1

plt.show()

In [ ]:
rnn_model.save_weights('model.h5')
cnn_model.save_weights('model.h51')